In [20]:
import os.path
import shelve
from pprint import pprint

import numpy as np
import scipy.optimize

In [2]:
shelf = 'route_opt.db'
if os.path.exists(shelf):
    with shelve.open(shelf) as db:
        data_1 = db['morning data']
        data_2 = db['afternoon data']
else:
    data_1 = {}
    data_2 = {}

In [16]:
route_1 = {'Hill St': [25, 40],
           'Hegeman Bridge Rd': [30, 50],
           'Rt. 40': [45, 60],
           'Sarles Ferry': [40, 50],
           'CR-113': [45, 60],
           'Stillwater Bridge Rd': [25, 40],
           'Stillwater': [25, 40],
           'Hill in Stillwater': [25, 40],
           'Rt. 4': [35, 45],
           'Mechanicville': [25, 40],
           'Leaving Mechanicville': [25, 45],
           '45 mph Rt. 4': [40, 55],
           'Rt. 146 hill': [40, 60],
           'Rt. 146 curve': [45, 60],
           'Rt. 146 before light': [45, 60],
           'Rt. 146 after light': [45, 55],
           'Rt. 146 Halfmoon': [45, 55],
           'Clifton Park': [40, 50],
           'Hill by Vent Fitness': [40, 60],
           '55 mph Rt. 146': [45, 60],
           'Rt. 146 near Balltown': [40, 60],
           'Balltown in Rexford': [40, 50],
           'Balltown on bridge': [40, 50],
           '45 mph Balltown': [40, 50],
           '40 mph Balltown': [40, 50],
           'River Rd': [35, 50],
          }

In [4]:
route_2 = [[35, 50],  # River Rd
           [40, 50],  # 40 mph Balltown
           [40, 50],  # 45 mph Balltown
           [40, 50],  # Balltown on bridge
           [40, 50],  # Balltown in Rexford
           [40, 60],  # Rt. 146 near Balltown
           [45, 60],  # 55 mph Rt. 146
           [40, 60],  # Hill by Vent Fitness
           [40, 50],  # Clifton Park
           [45, 55],  # Rt. 146 Halfmoon
           [45, 60],  # Rt. 146 before light
           [45, 60],  # Rt. 146 after light
           [45, 60],  # Rt. 146 curve
           [40, 60],  # Rt. 146 hill
           [40, 55],  # 45 mph Rt. 4
           [30, 45],  # Entering Mechanicville
           [25, 40],  # Mechanicville
           [35, 45],  # Rt. 4
           [25, 40],  # Hill in Stillwater
           [25, 40],  # Stillwater
           [25, 40],  # Stillwater Bridge Rd
           [45, 60],  # CR-113
           [40, 50],  # Sarles Ferry
           [45, 60],  # Rt. 40
           [30, 50],  # Hegeman Bridge Rd
           [25, 40],  # Hill St
          ]

In [17]:
x0_1 = [round(np.mean(bounds)) for bounds in route_1.values()]
x0_2 = [round(np.mean(bounds)) for bounds in route_2.values()]

AttributeError: 'list' object has no attribute 'values'

In [6]:
# with shelve.open(shelf) as db:
#     db['morning data'] = data_1
#     db['afternoon data'] = data_2

In [21]:
ITER = 0
NOT_FOUND = True
def f(x, dset=None):
    global ITER, NOT_FOUND
    data = globals()[f'data_{dset}']
    route = globals()[f'route_{dset}']
    x = tuple([ITER] + list(map(int, np.rint(x))))
    if NOT_FOUND and x in data:
        y = data[x]
        print(f'MPG for {x[1:]} = {y}')
    else:
        NOT_FOUND = False
        pprint({seg: val for seg, val in zip(route, x[1:])})
        y = float(input(f'Enter MPG for {x[1:]}\n--> '))
        data[x] = y
    ITER += 1
    return 10000 / y

In [22]:
res = scipy.optimize.minimize(f, x0_1, args=('1',), method='SLSQP',
                              options={'ftol': 1, 'disp': True, 'eps': 1}, bounds=route_1)

TypeError: float() argument must be a string or a number, not 'dict'

In [ ]:
list(map(int, np.rint([1.5, 2.5, 3.5])))